In [2]:
print("ok")

ok


In [7]:
%pwd

'/workspaces/Medical-Chatbot-Gen-AI/research'

In [2]:
import os
os.chdir("/workspaces/Medical-Chatbot-Gen-AI")
%pwd

'/workspaces/Medical-Chatbot-Gen-AI'

In [3]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
def load_pdf_file(data):
    loader= DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [5]:
extracted_data=load_pdf_file(data='Data/')

In [6]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [7]:
text_chunks=text_split(extracted_data)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

# Correct function call
embeddings = download_huggingface_embeddings()

/tmp/ipykernel_2457/348322451.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
query_result=embeddings.embed_query("Hello World")
print("length", len(query_result))

length 384


In [10]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GEMINI_API_KEY=os.environ.get('GEMINI_API_KEY')

In [12]:
pip install pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
pip show pinecone

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Name: pinecone
Version: 7.3.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/python/3.12.1/lib/python3.12/site-packages
Requires: certifi, pinecone-plugin-assistant, pinecone-plugin-interface, python-dateutil, typing-extensions, urllib3
Required-by: langchain-pinecone
Note: you may need to restart the kernel to use updated packages.


In [13]:
import sys
print(sys.path)
import pinecone
print(pinecone.__file__, pinecone.__version__)


['/usr/local/python/3.12.1/lib/python312.zip', '/usr/local/python/3.12.1/lib/python3.12', '/usr/local/python/3.12.1/lib/python3.12/lib-dynload', '', '/home/codespace/.local/lib/python3.12/site-packages', '/usr/local/python/3.12.1/lib/python3.12/site-packages', '/tmp/tmpebuyjwq7']
/usr/local/python/3.12.1/lib/python3.12/site-packages/pinecone/__init__.py 7.3.0


In [14]:
import pinecone
print(pinecone.__file__)   # should show .../site-packages/pinecone/__init__.py
print([name for name in dir(pinecone) if "Pinecone" in name])



/usr/local/python/3.12.1/lib/python3.12/site-packages/pinecone/__init__.py
['Pinecone', 'PineconeApiAttributeError', 'PineconeApiException', 'PineconeApiKeyError', 'PineconeApiTypeError', 'PineconeApiValueError', 'PineconeAsyncio', 'PineconeConfig', 'PineconeConfigurationError', 'PineconeException', 'PineconeProtocolError']


In [13]:
from pinecone import Pinecone
import os
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name= "medicalbot"


In [32]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY
os.environ["GEMINI_API_KEY"]=GEMINI_API_KEY



TypeError: str expected, not NoneType

In [16]:
from langchain_pinecone import PineconeVectorStore

index_name="medicalbot"
docsearch= PineconeVectorStore.from_documents(
    documents= text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [17]:
from langchain_pinecone import PineconeSparseVectorStore
docsearch=PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding= embeddings
)

In [18]:
docsearch

In [19]:
retriever=docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [31]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm=ChatGoogleGenerativeAI(model="gemini-2.5-pro")

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.